In [ ]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.5 MB/s eta 0:00:00


The code below pulls data from open meteo and exports the data to an excel file

In [ ]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 40.7143,
    "longitude": -74.006,
    "start_date": "2021-01-01",
    "end_date": "2023-12-30",
    "daily": ["temperature_2m_max", "temperature_2m_min", "temperature_2m_mean", "precipitation_sum", "rain_sum", "snowfall_sum"],
    "temperature_unit": "fahrenheit",
    "wind_speed_unit": "mph",
    "precipitation_unit": "inch",
    "timezone": "America/New_York"
}
responses = openmeteo.weather_api(url, params=params)

response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

daily = response.Daily()
daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_mean = daily.Variables(2).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(3).ValuesAsNumpy()
daily_rain_sum = daily.Variables(4).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(5).ValuesAsNumpy()

daily_data = {
    "date": pd.date_range(
        start=pd.to_datetime(daily.Time(), unit="s", utc=True),
        end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=daily.Interval()),
        inclusive="left"
    )
}
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["rain_sum"] = daily_rain_sum
daily_data["snowfall_sum"] = daily_snowfall_sum

daily_dataframe = pd.DataFrame(data=daily_data)

daily_dataframe['date'] = daily_dataframe['date'].dt.date
daily_dataframe['month'] = pd.DatetimeIndex(daily_dataframe['date']).month

numeric_columns = daily_dataframe.select_dtypes(include=['float64', 'int64']).columns
daily_dataframe[numeric_columns] = daily_dataframe[numeric_columns].round(-2)

print(daily_dataframe)


Coordinates 40.738136291503906°N -74.04254150390625°E
Elevation 51.0 m asl
Timezone b'America/New_York' b'EDT'
Timezone difference to GMT+0 -14400 s
            date  temperature_2m_max  temperature_2m_min  temperature_2m_mean  \
0     2021-01-01           38.340500           27.270500            33.401752   
1     2021-01-02           49.680500           34.380501            41.438000   
2     2021-01-03           38.340500           29.700500            34.545498   
3     2021-01-04           43.740501           33.030499            36.855503   
4     2021-01-05           40.770500           33.480499            36.308002   
...          ...                 ...                 ...                  ...   
1089  2023-12-26           45.450500           36.990501            40.301746   
1090  2023-12-27           47.880501           41.130501            43.980499   
1091  2023-12-28           52.650501           46.980499            49.459248   
1092  2023-12-29           55.710499     

In [ ]:
daily_dataframe.head(500)

,date,temperature_2m_max,temperature_2m_min,temperature_2m_mean,precipitation_sum,rain_sum,snowfall_sum,month
0,2021-01-01,38.340500,27.270500,33.401752,0.318898,0.318898,0.000000,1
1,2021-01-02,49.680500,34.380501,41.438000,0.161417,0.161417,0.000000,1
2,2021-01-03,38.340500,29.700500,34.545498,0.255906,0.251969,0.027559,1
3,2021-01-04,43.740501,33.030499,36.855503,0.059055,0.051181,0.055118,1
4,2021-01-05,40.770500,33.480499,36.308002,0.000000,0.000000,0.000000,1
...,...,...,...,...,...,...,...,...
495,2022-05-11,69.930496,47.520500,58.076756,0.000000,0.000000,0.000000,5
496,2022-05-12,69.570496,51.120499,59.479248,0.000000,0.000000,0.000000,5
497,2022-05-13,70.290497,56.700500,61.260509,0.070866,0.070866,0.000000,5
498,2022-05-14,69.030495,57.060501,61.598007,0.275591,0.275591,0.000000,5


In [ ]:
daily_data["temperature_2m_max"] = daily_temperature_2m_max.astype(float)
daily_data["temperature_2m_min"] = daily_temperature_2m_min.astype(float)
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean.astype(float)
daily_data["precipitation_sum"] = daily_precipitation_sum.astype(float)
daily_data["rain_sum"] = daily_rain_sum.astype(float)
daily_data["snowfall_sum"] = daily_snowfall_sum.astype(float)
daily_dataframe[numeric_columns] = daily_dataframe[numeric_columns].round(-2)


In [ ]:
daily_dataframe.head(500)

,date,temperature_2m_max,temperature_2m_min,temperature_2m_mean,precipitation_sum,rain_sum,snowfall_sum,month
0,2021-01-01,38.340500,27.270500,33.401752,0.318898,0.318898,0.000000,1
1,2021-01-02,49.680500,34.380501,41.438000,0.161417,0.161417,0.000000,1
2,2021-01-03,38.340500,29.700500,34.545498,0.255906,0.251969,0.027559,1
3,2021-01-04,43.740501,33.030499,36.855503,0.059055,0.051181,0.055118,1
4,2021-01-05,40.770500,33.480499,36.308002,0.000000,0.000000,0.000000,1
...,...,...,...,...,...,...,...,...
495,2022-05-11,69.930496,47.520500,58.076756,0.000000,0.000000,0.000000,5
496,2022-05-12,69.570496,51.120499,59.479248,0.000000,0.000000,0.000000,5
497,2022-05-13,70.290497,56.700500,61.260509,0.070866,0.070866,0.000000,5
498,2022-05-14,69.030495,57.060501,61.598007,0.275591,0.275591,0.000000,5


In [ ]:
daily_dataframe.to_excel('weather_data.xlsx', index=False)
